# LSTMega

Este projeto é um modelo de rede neural recorrente utilizando a arquitetura *Long Short-Term Memory* &ndash; LSTM para geração automatizada de textos.

Antes de mais nada, é importante ressaltar que este projeto estava baseado em uma implementação da biblioteca [Keras](https://keras.io/), a partir de um [exemplo](https://keras.io/examples/lstm_text_generation/) da utilização de redes neurais recorrentes com LSTM, disponível na própria documentação do Keras.

Porém, apesar dos vários ajustes no código-fonte, os resultados não estavam sendo satisfatórios. Então, para contornar esta situação, optou-se por utilizar outro algoritmo de geração de textos que, segundo a opinião de outros colegas, apresentava textos mais coesos e um melhor desempenho nos treinos.

Este algoritmo, apesar de também utilizar algumas funções da biblioteca Keras, foi produzido pelo [TensorFlow](https://www.tensorflow.org/) e apresenta uma aplicação de rede neural recorrente para geração de textos utilizando a arquitetura GRU, podendo ser facilmente adaptada para a LSTM. Este exemplo pode ser acessado [aqui](https://www.tensorflow.org/tutorials/text/text_generation).

No decorrer desta documentação, há alguns trechos de código de como este projeto estava antes de ser substituído pelo novo algoritmo. Todo o projeto e histórico de mudanças está disponível no repositório [lstmega](https://github.com/mlc2307/lstmega) no GitHub.

## Antes da alteração

Neste trecho da documentação será possível visualizar como a aplicação estava antes da troca de algoritmo.

Para manter um pouco da organização deste *notebook*, optou-se por não documentar o trecho de código que fazia as importações do algoritmo anterior. Além do mais, os dois exemplos são bem semelhantes nesta etapa.

Anteriormente, no primeiro bloco era informado o texto utilizado para o aprendizado do modelo, além de outras operações.

```python
# Origem remota do arquivo de texto utilizado no aprendizado do modelo.
origin = "https://s3.amazonaws.com/text-datasets/nietzsche.txt"

path = get_file(basename(origin), origin)

# Faz a leitura do arquivo, deixando todos os caracteres minúsculos.
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
    
print("Tamanho do arquivo:", len(text))

# Verifica quantos caracteres diferentes existem no texto.
chars = sorted(list(set(text)))

print("Total de caracteres:", len(chars))

# Faz a indexação dos caracteres.
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
```

A seguir, o algoritmo fatiava o texto em sequências semi-redundantes, cuja quantidade de caracteres destas sequências era definida pela variável `maxlen`.

```python
# Tamanho das fatias do texto.
maxlen = 40

step = 3

sentences = []
next_chars = []

# Processo responsável por fatiar o texto em sequências.
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i:i + maxlen])
    next_chars.append(text[i + maxlen])

print("Total de sequências:", len(sentences))

print("\nVetorizando as sequências...")

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        
    y[i, char_indices[next_chars[i]]] = 1

print("Sequências vetorizadas!")
```

Na próxima etapa vinha a definição e construção do modelo de rede neural.

```python
print("Construindo o modelo...")

model = Sequential()

# A camada de entrada é do tipo LSTM.
model.add(LSTM(128, input_shape=(maxlen, len(chars))))

# Adiciona uma camada do tipo Dense com 24 neurônios e 0.2 de dropout.
model.add(Dense(24, activation="relu"))
model.add(Dropout(0.2))

# Adiciona uma camada do tipo Dense com 32 neurônios e 0.3 de dropout.
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.3))

# Camada de saída.
model.add(Dense(len(chars), activation="softmax"))

print("Modelo construído!\n")

model.summary()
```

Logo após, duas funções eram definidas para auxiliarem nos processos de treinamento do modelo. A função `sample` amostrava um índice de uma matriz de probabilidade e a função `on_epoch_end` mostrava um *feedback* com algumas informações ao final de cada época.

```python
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    
    exp_preds = np.exp(preds)
    
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    print("\n\nGerando texto após época #{:d}...".format(epoch + 1))

    start_index = random.randint(0, len(text) - maxlen - 1)
    
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("\n{:->10s}".format(""))
        print("Diversidade:", diversity)

        generated = ""
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        
        print("Gerando com a seed \"" + sentence + "\"\n")
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        
        print()
    
    print("{:s}{:->20s}".format("\n" * 3, ""))

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
```

E por último era feita a configuração da compilação e treinamento deste modelo, onde vários parâmtros haviam sido modificados em relação ao exemplo do original.

```python
model.compile(
    loss="categorical_crossentropy",
    metrics=['accuracy'],
    optimizer=RMSprop()
)

model.fit(
    x, y,
    batch_size=96,
    epochs=64,
    callbacks=[print_callback]
)
```

A cada época era gerado um texto usando diferentes *diversidades*. Quanto maior a diversidade, maior era a "mistura" de caracteres empregada na geração, e, quanto mais avançada a época, mais coeso era o texto.

No final dos treinamentos, mesmo com vários ajustes, não eram obtidos bons resultados. Portanto, como já foi dito anteriormente, empregar um outro algoritmo pareceu uma solução interessante para este projeto.

O novo algoritmo que substituiu o anterior é apresentado daqui em diante.

## Preparos iniciais

### Importação das bibliotecas

A biblioteca base deste projeto é o TensorFlow, mas também possui algumas funções principais trazidas da biblioteca Keras, além de outras bibliotecas mais comuns.

In [ ]:
# Deve estar na primeira linha.
from __future__ import absolute_import, division, print_function, unicode_literals

# Ao importar o TensoFlow, seriam exibidos alguns avisos. A função abaixo os ignora.
import warnings
warnings.filterwarnings("ignore", module="tensorflow", category=FutureWarning)

# Biblioteca base.
import tensorflow as tf

# Ferramentas para manuseio de arquivos.
from os.path import basename, join
import io

# Para a geração dos arrays a partir do texto.
import numpy as np

# Usado na geração aleatória do seed.
import random

### *Download* do *dataset*

Para mudar um pouco o modo como as coisas acontecem, o arquivo de texto usado como *dataset* será o mesmo usado antes da alteração. O texto está disponível remotamente e pode ser acessado através deste [link](https://s3.amazonaws.com/text-datasets/nietzsche.txt).

In [ ]:
# Origem remota do arquivo de texto utilizado no aprendizado do modelo.
origin = "https://s3.amazonaws.com/text-datasets/nietzsche.txt"
filename = basename(origin)

# Retorna informações sobre o dataset.
print("O dataset é o arquivo \"{}\", disponível em {}".format(filename, origin))

# Faz o download do dataset.
path_to_file = tf.keras.utils.get_file(filename, origin)

### Leitura dos dados

Lê o arquivo de texto e obtém algumas informações.

In [ ]:
# Faz a leitura do arquivo de texto.
text = io.open(path_to_file, encoding="utf-8").read()

# Quantidade de caracteres no texto.
print("O texto possui {} caracteres".format(len(text)))

# Quantidade de caracteres únicos.
vocab = sorted(set(text))
print ("Existem {} caracteres únicos".format(len(vocab)))

## Processamento do texto

A seguir serão feitos alguns processamentos fundamentais com o texto do *dataset* baixado anteriormente.

### Vetorização do texto

Antes de treinar o modelo, é necessário mapear as string para uma representação numérica. Para isso serão criadas duas tabelas indexadas: uma que mapeia caracteres à números, e outra números à caracteres.

In [ ]:
# Mapeamento dos caracteres únicos à números.
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Depois do procedimento acima, cada caractere recebeu uma representação numérica. Abaixo há uma amostra para ilustrar isto de uma forma mais compreensível.

In [ ]:
# Imprime uma linha de dados de largura fixa.
def print_data(data, column_width=2):
    for i in data[:-1]:
        print("{:>{column_width}} ".format(i, column_width=column_width), end="")
    
    print("{:>{column_width}}".format(data[-1], column_width=column_width))

# Início e tamanho da frase.
phrase_begin = 54
phrase_len = 19

# Imprime os caracteres da frase e suas respectivas representações numéricas.
print_data(text[phrase_begin:phrase_begin + phrase_len])
print_data(text_as_int[phrase_begin:phrase_begin + phrase_len])

### Amostras de treinamento e alvos

Nesta etapa o texto é dividido em sequências de amostragem. Cada sequência conterá `seq_length` caracteres do texto.

Para cada sequência de entrada, a quantidade de caracteres das sequências alvo correspondentes será a mesma, mas com um deslocamento de um caractere para a esquerda.

Isso quer dizer que os textos estão sendo dividos em partes de tamanho `seq_length + 1`. Por exemplo, suponha-se que `seq_length` é 4 e o trecho de texto é "Tchau". Neste caso, a sequência de entrada seria "Tcha", e a sequência alvo "chau".

Para fazer isso, primeiro usa-se a função `tf.data.Dataset.from_tensor_slices` para converter o vetor de texto em uma indexação de caracteres.

In [ ]:
# O tamanho máximo de cada sequência.
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

# Cria as amostras de treinamento e os alvos.
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(7):
    print(idx2char[i.numpy()])

O método `batch` permite converte facilmente estes caracteres avulsos em sequências de um tamanho específico.

In [ ]:
# Cria os batches para o treinamento.
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

for item in sequences.take(7):
    print(repr("".join(idx2char[item.numpy()])))

Cada sequência é duplicada e deslocada para formar o texto de entrada e de destino usando o método `map` para aplicar uma simples função em cada *batch*.

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    
    return input_text, target_text

dataset = sequences.map(split_input_target)

Imprime a primeira sequência de amostragem e valores alvo.

In [ ]:
for input_example, target_example in dataset.take(1):
    print ("Dado de entrada:", repr("".join(idx2char[input_example.numpy()])))
    print ("Dado alvo:      ", repr("".join(idx2char[target_example.numpy()])))

Cada índice desses vetores é processado como uma etapa única. Para a entrada na etapa 0, o modelo recebe o índice para "P" e tenta prever o índice para "R" como o próximo caractere. No próximo passo, faz a mesma coisa, mas a RNN considera o contexto da etapa anterior além do caractere de entrada atual.

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:7], target_example[:7])):
    print("Passo {}:".format(i + 1))
    print("  Entrada: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  Saída esperada: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

### *Batches* de treinamento

Utiliza-se `tf.data` para dividir o texto em sequências gerenciáveis. Mas antes de alimentar esses dados no modelo, é necessário embaralhar os dados e agrupá-los em *batches*.

In [ ]:
# Tamanho do batch.
BATCH_SIZE = 64

# Tamanho do buffer para embaralhar o conjunto de dados.
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(dataset)

## Construção do modelo

O modelo que será construído é do tipo `tf.keras.Sequential`.

Para a construção deste modelo são utilizados três tipos de camadas:

* `tf.keras.layers.Embedding`: é a camada de entrada. Uma espécie de tabela que mapeará os números de cada caractere para um *array* com dimensões `embedding_dim`.
* `tf.keras.layers.LSTM`: uma arquitetura de rede neural recorrente, com tamanho `units=rnn_units`.
* `tf.keras.layers.Dense`: demais camadas e camada de saída, com `vocab_size` saídas.

In [ ]:
# Tamanho do vocabulário, em caracteres.
vocab_size = len(vocab)

# Dimensões da camada Embedding.
embedding_dim = 256

# Quantidade de unidades da camada LSTM.
rnn_units = 1024

A função `build_model` é responsável pela instanciação de um modelo de rede neural, dados os seus parâmetros.

Na primeira célula, seu algoritmo está igual ao do exemplo original.

In [ ]:
# Constrói um modelo dados os parâmetros.
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    # Modelo do tipo sequencial.
    model = tf.keras.Sequential()
    
    # Adiciona uma camada de entrada do tipo Embedding.
    model.add(tf.keras.layers.Embedding(vocab_size,
                                        embedding_dim,
                                        batch_input_shape=[batch_size, None]))
    
    # Adiciona a camada LSTM.
    model.add(tf.keras.layers.LSTM(rnn_units,
                                   return_sequences=True,
                                   stateful=True,
                                   recurrent_initializer='glorot_uniform'))
    
    # Última camada, do tipo Dense.
    model.add(tf.keras.layers.Dense(vocab_size))
    
    return model

Resta agora instanciar um novo modelo usando a recém-criada função `build_model`.

In [ ]:
# Instancia um novo modelo.
model = build_model(vocab_size,
                    embedding_dim,
                    rnn_units,
                    BATCH_SIZE)

## Teste do modelo

Nesta etapa, o modelo é executado para verificar se tudo se comporta como o esperado.

In [ ]:
# Verifica o shape de saída do modelo.
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    indexes_batch_predictions = ("batch_size", "sequence_length", "vocab_size")
    
    for i, e in zip(indexes_batch_predictions, example_batch_predictions.shape):
        print("{}: {}".format(i, e))

No exemplo acima, o comprimento da sequência da entrada `sequence_length` é `100`, mas o modelo pode ser executado em entradas de qualquer comprimento, conforme célula abaixo.

In [ ]:
# Exibe informações sobre o modelo.
model.summary()

Para obter previsões reais do modelo, precisamos fazer amostras a partir da distribuição de saída, para obter índices reais de caracteres. Essa distribuição é definida pelos *logits* sobre o vocabulário dos caracteres.

Nota: é importante fazer uma amostra dessa distribuição, pois o *argmax* da distribuição pode facilmente deixar o modelo preso em um *loop*.

In [ ]:
# Faz uma amostragem da distribuição.
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

Isso dá, a cada passo, uma previsão do próximo índice de caracteres.

In [ ]:
# Imprime os índices dos caracteres amostrados.
print(sampled_indices)

Ao decodificar os índices é possível visualizar o texto previsto por este modelo que ainda não foi treinado.

In [ ]:
# Decodifica os índices amostrados em sampled_indices.
print("Entrada:\n ", repr("".join(idx2char[input_example_batch[0]])))
print("\nPredições para o próximo caractere:\n ", repr("".join(idx2char[sampled_indices])))

## Treinamento do modelo

Neste ponto, o problema pode ser tratado como um problema padrão de classificação. Dado o estado anterior da RNN e a entrada em tal intervalo de tempo, deve ser prevista, então, a classe do próximo caractere.

### Otimizador e função de *loss*

A função padrão de *loss* `tf.keras.losses.sparse_categorical_crossentropy` funciona muito bem neste caso porque ela é aplicada na última dimensão das predições.

Nota: como o modelo retorna *logits*, é necessário definir a *flag* `from_logits`.

In [ ]:
# Função de loss.
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Aplica a função de loss no exemplo.
example_batch_loss = loss(target_example_batch, example_batch_predictions)

for i, e in zip(indexes_batch_predictions, example_batch_predictions.shape):
    print("{}: {}".format(i, e))

print("scalar_loss:", example_batch_loss.numpy().mean())

O próximo passo é configurar o treinamento usando a função `tf.keras.Model.compile`.

O exemplo original utiliza o otimizador `tf.keras.optimizers.Adam`, com seus argumentos padrão, e a função de *loss* definida anteriormente.

In [ ]:
# Configura o treinamento do modelo.
model.compile(optimizer="adam",
              loss=loss)

### Configuração dos *checkpoints*

Utiliza-se a função `tf.keras.callbacks.ModelCheckpoint` para garantir que os *checkpoints* sejam salvos durante o treinamento, conforme a célula a seguir.

In [ ]:
# Diretório onde os checkpoints serão salvos.
checkpoint_dir = './checkpoints'

# Nome dos arquivos dos checkpoints.
checkpoint_prefix = join(checkpoint_dir, "ckpt_{epoch}")

# Define o callback para cada época.
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                                       save_weights_only=True)

### Execução do treinamento

Na próxima célula é definida a quantidade de épocas do treinamento.

O exemplo original do TensorFlow define apenas 10 épocas para manter o treinamento mais razoável. Para este projeto, de início será definida uma quantidade de 20 épocas.

In [ ]:
# Define a quantidade de épocas.
EPOCHS = 20

A função `fit` do modelo é responsável por executar o treinamento. Após a execução ela retorna um histórico, que pode ser manipulado.

In [ ]:
# Executa o treinamento.
history = model.fit(dataset,
                    epochs=EPOCHS,
                    callbacks=[checkpoint_callback])

## Geração do texto

Diferentemente do processo de geração de texto que o algoritmo anterior tinha, o exemplo original deste algoritmo executa a geração de texto apenas após todo o treinamento ser concluído, ou seja, o texto não é gerado a cada época.

### O *loop* de predição

A lógica do funcionamento da geração de texto é simples:

* O algoritmo começa escolhendo uma sequência de início (*seed*), inicializando o estado RNN e definindo o número de caracteres a serem gerados.

* Depois é obtida a distribuição de predição do próximo caractere usando a sequência de início e o estado RNN.

* Em seguida, é usada uma distribuição categórica para calcular o índice do caractere previsto. Feito isto, este caractere passa a ser a próxima entrada do modelo.

* O estado RNN retornado pelo modelo é realimentado no modelo para que agora tenha mais contexto, em vez de apenas uma palavra. Depois de prever a próxima palavra, os estados RNN modificados são novamente alimentados no modelo. É assim que a rede neural aprende à medida que obtém mais contexto das palavras previstas anteriormente.

![Para gerar o texto, uma saída do modelo alimenta a entrada seguinte](images/text_generation_sampling.png)

A função `generate_text` mostrada abaixo é responsável pela geração de texto e é utilizada no exemplo original.

```python
# Gera um texto usando o modelo treinado.
def generate_text(model, start_string):
    # Número de caracteres do texto.
    num_generate = 1000

    # Vetoriza a seed, convertendo os caracteres para números.
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Variável para armazenar os resultados.
    text_generated = []

    # Baixas temperaturas resultam em um texto mais previsível.
    # Temperaturas mais altas resultam em um texto mais surpreendente.
    temperature = 1.0

    # No exemplo original, a função abaixo redefine os estados
    # do modelo para que tenha um batch_size igual à 1.
    # Isto não é utilizado neste projeto.
    model.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_eval)
        
        # Remove a dimensão do batch.
        predictions = tf.squeeze(predictions, 0)

        # Usa uma distribuição categórica para predizer a palavra retornada pelo modelo.
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Realimenta a próxima entrada do modelo com a palavra prevista,
        # juntamente com o estado oculto anterior.
        input_eval = tf.expand_dims([predicted_id], 0)
        
        text_generated.append(idx2char[predicted_id])
    
    return (start_string + ''.join(text_generated))
```

Para ampliar a gama de resultados deste projeto, algumas pequenas modificações foram feitas na função `generate_text` e podem ser visualizadas a seguir. Aliás, parte desta modificações são do algoritmo anterior e inclui, principalmente, *seeds* aleatórias e mais opções de temperaturas, além de outras funcionalidades.

In [ ]:
# Gera um texto usando o modelo treinado.
def generate_text(model, temperatures):
    # Número de caracteres do texto.
    num_generate = 1000
    
    # Armazenará todos os textos gerados.
    texts = []
    
    # Obtém um começo aleatório para a seed.
    start_index = random.randint(0, len(text) - seq_length - 1)
    
    # Forma a seed aleatória a partir do caractere na posição
    # start_index do texto e de comprimento seq_length.
    start_string = text[start_index:start_index + seq_length]
    
    # Vetoriza a seed, convertendo os caracteres para números.
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    # Baixas temperaturas resultam em um texto mais previsível.
    # Temperaturas mais altas resultam em um texto mais embaralhado.
    for temperature in temperatures:
        # Variável para armazenar os resultados.
        text_generated = []
        
        # Faz a predição dos caracteres.
        for i in range(num_generate):
            predictions = model(input_eval)

            # Remove a dimensão do batch.
            predictions = tf.squeeze(predictions, 0) / temperature

            # Usa uma distribuição categórica para predizer a palavra retornada pelo modelo.
            predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

            # Realimenta a próxima entrada do modelo com a palavra prevista,
            # juntamente com o estado oculto anterior.
            input_eval = tf.expand_dims([predicted_id], 0)

            text_generated.append(idx2char[predicted_id])

        texts.append({
            "temperature": temperature,
            "text_generated": start_string + ''.join(text_generated)
        })
    
    # Retorna todos os textos gerados.

Agora basta chamar a função `generate_text` e informar seus parâmetros para que os textos sejam gerados. Além disso, a célula abaixo imprime os textos gerados com base nas temperaturas definidas na variável `temperatures`.

In [ ]:
# Temperaturas que serão utilizadas na geração dos textos.
temperatures = [0.5, 0.6, 0.7, 0.8, 0.9]

# Finalmente, executa a geração dos textos.
texts = generate_text(model, temperatures)

# Imprime os textos gerados.
for text in texts:
    print("Temperatura: {}\n\n{}".format(text["temperature"], text["text_generated"]))
    
    # Imprime um separador entre os textos.
    if text["temperature"] != temperatures[-1]:
        print("{:->20s}".format(""))